# Slack

本笔记本演示了如何使用 Slack chat loader。此类有助于将导出的 Slack 对话映射到 LangChain 聊天消息。

该过程有三个步骤：
1. 按照[此处说明](https://slack.com/help/articles/1500001548241-Request-to-export-all-conversations)导出所需的对话线程。
2. 使用指向 json 文件或 json 文件目录的文件路径创建 `SlackChatLoader`。
3. 调用 `loader.load()`（或 `loader.lazy_load()`）以执行转换。可选择使用 `merge_chat_runs` 来合并来自同一发送者的连续消息，以及/或使用 `map_ai_messages` 将来自指定发送者的消息转换为“AIMessage”类。

## 1. 创建消息转储

当前（2023 年 8 月 23 日），此加载器最好支持从 Slack 导出直接消息对话时生成的格式的文件的 zip 目录。请按照 Slack 的最新说明进行操作。

我们在 LangChain 代码库中有一个示例。

In [1]:
import requests

permalink = "https://raw.githubusercontent.com/langchain-ai/langchain/342087bdfa3ac31d622385d0f2d09cf5e06c8db3/libs/langchain/tests/integration_tests/examples/slack_export.zip"
response = requests.get(permalink)
with open("slack_dump.zip", "wb") as f:
    f.write(response.content)

## 2. 创建聊天加载器

向加载器提供 zip 目录的文件路径。您可以选择指定映射到 AI 消息的用户 ID，以及配置是否合并消息运行。

In [2]:
from langchain_community.chat_loaders.slack import SlackChatLoader

In [3]:
loader = SlackChatLoader(
    path="slack_dump.zip",
)

## 3. 加载消息

`load()` (或 `lazy_load`) 方法返回一个 "ChatSessions" 列表，目前该列表仅包含已加载对话的消息列表。

In [4]:
from typing import List

from langchain_community.chat_loaders.utils import (
    map_ai_messages,
    merge_chat_runs,
)
from langchain_core.chat_sessions import ChatSession

raw_messages = loader.lazy_load()
# Merge consecutive messages from the same sender into a single message
merged_messages = merge_chat_runs(raw_messages)
# Convert messages from "U0500003428" to AI messages
messages: List[ChatSession] = list(
    map_ai_messages(merged_messages, sender="U0500003428")
)

### 后续步骤

然后，你可以根据需要使用这些消息，例如用于微调模型、少样本示例选择，或直接对下一条消息进行预测。

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

for chunk in llm.stream(messages[1]["messages"]):
    print(chunk.content, end="", flush=True)

Hi, 

I hope you're doing well. I wanted to reach out and ask if you'd be available to meet up for coffee sometime next week. I'd love to catch up and hear about what's been going on in your life. Let me know if you're interested and we can find a time that works for both of us. 

Looking forward to hearing from you!

Best, [Your Name]